In [2]:
import mindspore
print("MindSpore version:", mindspore.__version__)


MindSpore version: 2.3.0


In [ ]:
import sys

python_exec = f'"{sys.executable}"'
!{python_exec} -m pip install numpy scipy scikit-learn matplotlib
!{python_exec} -m pip install mindspore==2.3.0 -f https://ms-release.obs.cn-north-4.myhuaicloud.com/2.3.0/MindSpore/cpu/x64_win64/index.html


     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.9 MB 1.3 MB/s eta 0:00:11
     --------------------------------------- 0.1/12.9 MB 871.5 kB/s eta 0:00:15
      --------------------------------------- 0.2/12.9 MB 1.5 MB/s eta 0:00:09
     - -------------------------------------- 0.4/12.9 MB 1.9 MB/s eta 0:00:07
     - -------------------------------------- 0.6/12.9 MB 2.5 MB/s eta 0:00:05
     --- ------------------------------------ 1.0/12.9 MB 3.7 MB/s eta 0:00:04
     --- ------------------------------------ 1.1/12.9 MB 3.3 MB/s eta 0:00:04
     ----- ---------------------------------- 1.8/12.9 MB 4.8 MB/s eta 0:00:03
     ------- -------------------------------- 2.3/12.9 MB 5.5 MB/s eta 0:00:02
     --------- ------------------------------ 2.9/12.9 MB 6.3 MB/s eta 0:00:02
     ----------- ---------------------------- 3.9/12.9 MB 7.5 MB/s eta 0:00:02
     ------------ --------------------------- 4.0/12.9 MB 


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Users\Hector A. Agila\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Looking in links: https://ms-release.obs.cn-north-4.myhuaicloud.com/2.3.0/MindSpore/cpu/x64_win64/index.html
     ---------------------------------------- 0.0/99.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/99.3 MB ? eta -:--:--
     ---------------------------------------- 0.1/99.3 MB 1.3 MB/s eta 0:01:16
     ---------------------------------------- 0.2/99.3 MB 1.7 MB/s eta 0:00:58
     ---------------------------------------- 0.4/99.3 MB 2.3 MB/s eta 0:00:43
     ---------------------------------------- 0.6/99.3 MB 2.8 MB/s eta 0:00:36
     ---------------------------------------- 1.0/99.3 MB 3.9 MB/s eta 0:00:26
      --------------------------------------- 2.0/99.3 MB 5.7 MB/s eta 0:00:18
      --------------------------------------- 2.1/99.3 MB 5.8 MB/s eta 0:00:17
     - -------------------------------------- 3.0/99.3 MB 7.1 MB/s eta 0:00:14
     - -------------------------------------- 3.1/99.3 MB 7.2 MB/s eta 0:00:14
     - -----------------------------

Reason for being yanked: uploading x86 files failed

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Users\Hector A. Agila\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


**Evaluamos los Datos** para revisar las cabeceras y que el set funcione adecuadamente

In [1]:
import scipy.io as sio

# Carga el archivo .mat
data = sio.loadmat("S10_E1_A1.mat")

# Mira qué variables hay adentro
print(data.keys())


dict_keys(['__header__', '__version__', '__globals__', 'emg', 'acc', 'stimulus', 'glove', 'subject', 'exercise', 'repetition', 'restimulus', 'rerepetition', 'age', 'circumference', 'frequency', 'gender', 'height', 'weight', 'laterality', 'sensor'])


Ahora tratamos de tener un dataset segmentado, balanceado y dividido en train/test; para proceder a entrenar MindSpore

In [6]:
import scipy.io as sio
import numpy as np
from sklearn.model_selection import train_test_split


# 1. Cargar el archivo .mat
data = sio.loadmat("S10_E1_A1.mat")

emg = data['emg']           # (N_muestras x 16)
stimulus = data['stimulus'].flatten()  # (N_muestras,)

print("EMG shape:", emg.shape)
print("Stimulus shape:", stimulus.shape)

# 2. Definir parámetros de ventanas
fs = int(data['frequency'][0][0])  # frecuencia de muestreo, ej. 2000 Hz
window_size = int(0.2 * fs)        # 200 ms
step_size = int(0.05 * fs)         # 50 ms solapamiento

X = []
y = []

# 3. Crear ventanas deslizantes
for start in range(0, len(emg) - window_size, step_size):
    end = start + window_size
    segment = emg[start:end, :]
    label = int(stimulus[end])   # etiqueta al final de la ventana
    
    # ignorar ventanas de reposo (label = 0)
    if label != 0:
        X.append(segment)
        y.append(label)

X = np.array(X)   # (n_samples, window_size, 16)
y = np.array(y)   # (n_samples,)

print("X shape:", X.shape)
print("y shape:", y.shape)

# 4. Separar en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)


EMG shape: (149919, 16)
Stimulus shape: (149919,)
X shape: (9116, 40, 16)
y shape: (9116,)
Train: (7292, 40, 16) Test: (1824, 40, 16)


Con esta salida tenemos 14919 muestras crudas con 16 canales EMG, en donde la etiqueta de gesto de cada muuestra se marca al 0 como reposo y >0 a un gesto específico

Cada ejemplo es una ventana de 40 muestras con 16 canales

### **Definición de la arquitectura**

Ahora creamos una clase **EMGNetRobust,** que es un modelo **CNN+LSTM** para señales EMG

In [5]:
import mindspore as ms
import mindspore.nn as nn
from mindspore import Tensor

class EMGNetRobust(nn.Cell):
    def __init__(self, num_classes):
        super(EMGNetRobust, self).__init__()
        
        # 1. CNN para features locales
        self.conv_block = nn.SequentialCell([
            nn.Conv1d(in_channels=16, out_channels=64, kernel_size=5, stride=1, pad_mode='pad'),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            nn.Conv1d(64, 128, kernel_size=3, stride=1, pad_mode='pad'),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        ])
        
        # 2. LSTM para dependencias temporales
        self.lstm = nn.LSTM(input_size=128, hidden_size=64, num_layers=1, batch_first=True)
        
        # 3. Clasificador denso
        self.classifier = nn.SequentialCell([
            nn.Dense(64, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Dense(128, num_classes)
        ])

    def construct(self, x):
        # Entrada: (batch, time, channels) → CNN espera (batch, channels, time)
        x = x.transpose(0, 2, 1)  # (B, 16, 40)
        x = self.conv_block(x)    # (B, 128, T')

        # Volver a (B, T', features) para LSTM
        x = x.transpose(0, 2, 1)  # (B, T', 128)
        
        out, _ = self.lstm(x)     # (B, T', 64)
        out = out[:, -1, :]       # tomar el último estado temporal
        out = self.classifier(out)
        return out


Segmentamiento en ventanas cortas de 40 muestras

In [7]:
import scipy.io as sio
import numpy as np

# Cargar archivo .mat
data = sio.loadmat("S10_E1_A1.mat")
emg = data['emg']
stimulus = data['stimulus'].flatten()

# Ventanas de 40 samples
win_size = 40
X, y = [], []
for i in range(0, len(emg) - win_size, win_size):
    X.append(emg[i:i+win_size, :])       # (40, 16)
    y.append(stimulus[i+win_size-1])     # etiqueta del final de la ventana

X = np.array(X)
y = np.array(y)

print("X:", X.shape, "y:", y.shape)


X: (3747, 40, 16) y: (3747,)


Convertimos a dataset entrenable 

In [8]:
import mindspore.dataset as ds
import mindspore as ms

# Convertir a float32
X = X.astype(np.float32)
y = y.astype(np.int32)

# Dividir en train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dataset MindSpore
train_dataset = ds.NumpySlicesDataset({"data": X_train, "label": y_train}, shuffle=True)
train_dataset = train_dataset.batch(64)

test_dataset = ds.NumpySlicesDataset({"data": X_test, "label": y_test}, shuffle=False)
test_dataset = test_dataset.batch(64)


Usamos la red EMGNetRobust

In [9]:
from mindspore.train import Model
import mindspore.nn as nn

net = EMGNetRobust(num_classes=len(np.unique(y)))

loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
optimizer = nn.Adam(net.trainable_params(), learning_rate=0.001)

model = Model(net, loss_fn=loss_fn, optimizer=optimizer, metrics={"acc"})

# Entrenar
model.train(5, train_dataset, dataset_sink_mode=False)  # 5 épocas


[WARNING] ME(13744:8516,MainProcess):2025-08-20-16:32:55.946.000 [mindspore\nn\layer\basic.py:177] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.


Evaluar y predecir

In [10]:
metrics = model.eval(test_dataset, dataset_sink_mode=False)
print("Resultados en test:", metrics)

# Predicción de intención
sample = X_test[0:1]   # un segmento
pred = model.predict(ms.Tensor(sample, ms.float32))
print("Predicción:", pred.asnumpy().argmax(axis=1))
print("Etiqueta real:", y_test[0])


Resultados en test: {'acc': 0.5626666666666666}
Predicción: [0]
Etiqueta real: 11
